In [10]:
import requests
import json
from base64 import b64encode
import tkinter as tk
import random
from bs4 import BeautifulSoup

In [2]:
client_id = "e291792abd4b4a1db646f561c823beb9"
client_secret = "26e6df1ae3614825940739dd03c31feb"

auth_url = 'https://accounts.spotify.com/api/token'
auth_header = 'Basic ' + b64encode((client_id + ':' + client_secret).encode()).decode()
url = "https://api.spotify.com/v1/search"

auth_data = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post(auth_url, headers={'Authorization': auth_header}, data=auth_data)

if auth_response.status_code == 200:
    token = auth_response.json().get('access_token')
    print('Token:', token)
else:
    print('Error:', auth_response.status_code)
    print(auth_response.text)





NameError: name 'b64encode' is not defined

In [1]:
token = "BQDLS51gIeE-tdAjx3S78yTN_8nus2WYiwXYS_tQE6ktSSIp2A-DaB8pwY-7Gm6iRnXOfkfGM3LmkB6LU-j07Vm3TRvAYmUUgVdAie09YVmUqwqrUtevOQAfvuBkF74RJQGnnECTNbs"

#artist_name = "Taylor Swift"

def get_spotify_id(artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1  
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        results = response.json()
        if results["artists"]["items"]:
            artist = results["artists"]["items"][0]
            id = artist["id"]
            return id
        else:
            return "No artist found."
    else:
        return f"Error: {response.status_code} - {response.text}"

In [11]:
API_KEY = "OdOTpaxGkQyamSaW2EkeanimV3vOKKdmyBTwQHGHxErCN97F3c1Z4kVR8UJWgM85"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def get_lyrics(artist):
    response = requests.get("https://api.genius.com/search", headers=HEADERS, params={"q": artist})
    data = response.json()

    if not data["response"]["hits"]:
        print("No songs found.\n")
        return []
    else:
        song = random.choice(data["response"]["hits"])["result"]
        
        title = song["title"]
        
        hint = song["title"][0]

        page = requests.get(song["url"])
        soup = BeautifulSoup(page.text, "html.parser")

        lyrics_divs = soup.find_all("div", class_=lambda x: x and "Lyrics__Container" in x)

        full_lyrics = ""
        
        for i in lyrics_divs:
            full_lyrics += i.text.strip()

        words = full_lyrics.split() 

        if len(words) >= 10:
            stop = random.randint(10, len(words))
            start = stop - 10
            consecutive_words = words[start:stop]
            song_lst = [title, " ".join(consecutive_words), hint]
            return song_lst
                
        else:
            print("Not enough words in the lyrics.\n")
            return []

def q2():

    score = 0

    #question 1
    response_2 = input("Enter an artist's name: ")
    question_1 = get_lyrics(response_2)

    response_3 = input(f"Enter the song title for the following lyrics: {question_1[1]}\n")
    if response_3.lower() == question_1[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_1[0]}. Your current score is {score}/30\n")

    #question 2
    question_2 = get_lyrics(response_2)
    response_5 = input(f"Enter the song title for the following lyrics: {question_2[1]}\n")
    if response_5.lower() == question_2[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_2[0]}. Your current score is {score}/30\n")

    #question 3
    question_3 = get_lyrics(response_2)
    response_7 = input(f"Enter the song title for the following lyrics: {question_3[1]}\n")
    if response_7.lower() == question_3[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_3[0]}. Your current score is {score}/30\n")

    return f"Your score is {score}/30."

In [12]:
q2()


Correct! Your score is 10/30.


Correct! Your score is 20/30.


Incorrect. The correct answer was All Too Well (10 Minute Version) (Taylor’s Version) [Live Acoustic]. Your current score is 20/30



'Your score is 20/30.'

In [13]:
#root = tk.Tk()
#root.geometry("500x500")

#root.mainloop()